**DATE**: 13 November 2017 <br>
**ENVIRONMENT**: qiime2_mental_illness <br>
**AUTHOR**: Max Abramson, Tomasz Kosciolek

# This notebook analyses AGP participants who endorsed mental health problems, as reported in "A living dataset" section of the American Gut manuscript

In [1]:
import pandas as pd

In [2]:
# INPUTS
root_inp = 'data/mental_illness/'
original_table=root_inp+'mental_illness_250.biom'
original_tree=root_inp+'ag_tree.tree'
metadata = root_inp+'AGP_metadata_mental_illness_250.txt'

# OUTPUTS
root_out = 'data/mental_illness/out/'

table_all = root_out+'feature_table_raw.qza'
table_rarefied = root_out+'feature_table_rare1397.qza'
tree_artifact = root_out+'rooted_tree.qza'

metadata_US = root_out+'US_metadata.txt'
metadata_UK = root_out+'UK_metadata.txt'
metadata_women = root_out+'women_metadata.txt'
metadata_young = root_out+'young_metadata.txt'

beta_all = root_out+'weightedUniFrac_all.qza'

sign_all = root_out+'all_weighted_unifrac_case_significance.qzv'
sign_US = root_out+'US_weighted_unifrac_case_significance.qzv'
sign_UK = root_out+'UK_weighted_unifrac_case_significance.qzv'
sign_women = root_out+'women_weighted_unifrac_case_significance.qzv'
sign_young = root_out+'young_weighted_unifrac_case_significance.qzv'

## Import raw data as QIIME2 artifacts

In [3]:
!qiime tools import \
  --input-path $original_table \
  --type FeatureTable[Frequency] \
  --source-format BIOMV210Format \
  --output-path $table_all

In [4]:
!qiime tools import \
  --input-path $original_tree \
  --output-path $tree_artifact \
  --type 'Phylogeny[Rooted]'

Read metadata into a `pd.DataFrame`

In [5]:
mf = pd.read_csv(metadata, sep='\t', dtype=str).set_index('sample_name')

## Rarefy all tables

In [6]:
# rarefy at 1397 to retain all samples
!qiime feature-table rarefy \
  --i-table $table_all \
  --p-sampling-depth 1397 \
  --o-rarefied-table $table_rarefied

Saved FeatureTable[Frequency] to: data/mental_illness/out/feature_table_rare1397.qza


# ALL samples

## Run weighted UniFrac calculation


In [7]:
!qiime diversity beta-phylogenetic \
  --i-table $table_rarefied \
  --i-phylogeny $tree_artifact \
  --p-metric weighted_unifrac \
  --o-distance-matrix $beta_all

Saved DistanceMatrix % Properties(['phylogenetic']) to: data/mental_illness/out/weightedUniFrac_all.qza


## Beta diversity analysis

### Weighted UniFrac distance matrix specifically looking at 'case'

In [8]:
!qiime diversity beta-group-significance \
  --i-distance-matrix $beta_all \
  --m-metadata-file $metadata \
  --m-metadata-category case_control \
  --o-visualization $sign_all \
  --p-pairwise

Saved Visualization to: data/mental_illness/out/all_weighted_unifrac_case_significance.qzv


# USA-only samples

In [9]:
mf_usa = mf[mf['country_residence'] == 'USA']
mf_usa.to_csv(metadata_US, sep='\t')

In [10]:
!qiime diversity beta-group-significance \
  --i-distance-matrix $beta_all \
  --m-metadata-file $metadata_US \
  --m-metadata-category case_control \
  --o-visualization $sign_US \
  --p-pairwise

Saved Visualization to: data/mental_illness/out/US_weighted_unifrac_case_significance.qzv


# UK-only samples

In [11]:
mf_uk = mf[mf['country_residence'] == 'UK']
mf_uk.to_csv(metadata_UK, sep='\t')

In [12]:
!qiime diversity beta-group-significance \
  --i-distance-matrix $beta_all \
  --m-metadata-file $metadata_UK \
  --m-metadata-category case_control \
  --o-visualization $sign_UK \
  --p-pairwise

Saved Visualization to: data/mental_illness/out/UK_weighted_unifrac_case_significance.qzv


# Women-only samples

In [13]:
mf_women = mf[mf['sex'] == 'female']
mf_women.to_csv(metadata_women, sep='\t')

In [14]:
!qiime diversity beta-group-significance \
  --i-distance-matrix $beta_all \
  --m-metadata-file $metadata_women \
  --m-metadata-category case_control \
  --o-visualization $sign_women \
  --p-pairwise

Saved Visualization to: data/mental_illness/out/women_weighted_unifrac_case_significance.qzv


# Age <= 45 samples

In [15]:
mf_young = mf[mf['age_years'].astype(int) <= 45]
mf_young.to_csv(metadata_young, sep='\t')

In [16]:
!qiime diversity beta-group-significance \
  --i-distance-matrix $beta_all \
  --m-metadata-file $metadata_young \
  --m-metadata-category case_control \
  --o-visualization $sign_young \
  --p-pairwise

Saved Visualization to: data/mental_illness/out/young_weighted_unifrac_case_significance.qzv


# Visualize

ALL

In [17]:
!qiime tools view $sign_all

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

USA-only

In [18]:
!qiime tools view $sign_US

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

UK-only

In [19]:
!qiime tools view $sign_UK

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Women

In [20]:
!qiime tools view $sign_women

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

<= 45 years

In [21]:
!qiime tools view $sign_young

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.